CNN mit EfficientNet

In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1' #Kommentieren Sie diese Zeile aus, falls Sie ihre GPU benutzen können.

In [5]:
import tensorflow as tf
import tensorflow_hub as hub
import keras

from functions import create_tf_dataSet
from functions import upload_tensorboard
dataset_path = r"C:\Users\nilss\Sets\DataSet_v4 _train_aug"

IMG_SIZE = (224,224)

train_set = create_tf_dataSet(
    path=dataset_path+"/train",
    img_size=IMG_SIZE,
    batch_size=32
    )

validation_set = create_tf_dataSet(
    path=dataset_path+"/validation",
    img_size=IMG_SIZE,
    batch_size=32
)    



Found 26948 files belonging to 2 classes.
Found 6648 files belonging to 2 classes.


In [6]:
import tensorflow as tf
from keras.applications import EfficientNetB0
from keras import layers
import tensorflow as tf
from keras.applications import EfficientNetB0
from keras import layers
from keras import backend as K
import tensorflow as tf


input_shape = (224, 224, 3)
efficientNet_layer = tf.keras.applications.EfficientNetB3(include_top=False) # iclude_top=False, spezialisierte Layer werden entfernt.
efficientNet_layer.trainable = False # Die Gewichte sindgefrohren, sie sind nämlich schon sehr fein abgestimmt.

# Input Layer erstellen
inputs = tf.keras.layers.Input(shape=input_shape, name="input_layer")


# Daten werden direkt an EfficientNet weitergegeben, es findet keine Data Augmentation mit der Keras API statt.
# Für EfficientNet ist keine Normalisation notwendig.
x= efficientNet_layer(inputs)
# Pooling zur Verringerung der Parameter.

x = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)

# Für diesen Feature-Extractor, kommt direkt nach EfficientNet bereits das Output-Neuron.
outputs = tf.keras.layers.Dense(1, activation="sigmoid", name="output_layer",)(x)

# Erstellen des Modells.
efficientNet_model = tf.keras.Model(inputs, outputs)




# def focal_loss(gamma=2., alpha=.25):
#	 def focal_loss_fixed(y_true, y_pred):
#		 pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
#		 pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
#		 return -K.mean(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1+K.epsilon())) - K.mean((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0 + K.epsilon()))
#	 return focal_loss_fixed
# aus https://github.com/mkocabas/focal-loss-keras/blob/master/focal_loss.py


efficientNet_model.compile(
    loss="binary_crossentropy",
        
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    
    metrics=
            # [keras.metrics.Precision(thresholds=[0.253]),keras.metrics.Recall(thresholds=[0.253]),]
            [tf.keras.metrics.Recall(),tf.keras.metrics.Precision()],
                
)




                    


In [4]:
efficientNet_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb3 (Functional)  (None, None, None, 1536)  10783535 
                                                                 
 global_average_pooling_laye  (None, 1536)             0         
 r (GlobalAveragePooling2D)                                      
                                                                 
 output_layer (Dense)        (None, 1)                 1537      
                                                                 
Total params: 10,785,072
Trainable params: 1,537
Non-trainable params: 10,783,535
_________________________________________________________________


In [5]:
#Hier wird das Modell trainiert.
history = efficientNet_model.fit(
                    train_set,
                    epochs=20,
                    steps_per_epoch=len(train_set),
                    validation_data=validation_set,
                    validation_steps=int(len(validation_set)),
                    class_weight={0:1,1:10},
                    
                    callbacks=[upload_tensorboard('',''),] 
                    )

               
           


Epoch 1/20
843/843 [==============================] - 1367s 2s/step - loss: 0.6045 - precision: 0.1451 - recall: 0.3564 - val_loss: 0.1898 - val_precision: 0.0963 - val_recall: 0.2719
Epoch 2/20
843/843 [==============================] - 1880s 2s/step - loss: 0.5437 - precision: 0.1698 - recall: 0.4766 - val_loss: 0.2274 - val_precision: 0.0911 - val_recall: 0.3947
Epoch 3/20
843/843 [==============================] - 2486s 3s/step - loss: 0.5227 - precision: 0.1806 - recall: 0.5085 - val_loss: 0.1918 - val_precision: 0.1105 - val_recall: 0.3509
Epoch 4/20
843/843 [==============================] - 2492s 3s/step - loss: 0.4999 - precision: 0.1964 - recall: 0.5521 - val_loss: 0.1742 - val_precision: 0.1164 - val_recall: 0.3246
Epoch 5/20
843/843 [==============================] - 2488s 3s/step - loss: 0.4875 - precision: 0.2017 - recall: 0.5691 - val_loss: 0.1778 - val_precision: 0.1079 - val_recall: 0.3246
Epoch 6/20
843/843 [==============================] - 2498s 3s/step - loss: 0.48

In [7]:
efficientNet_model.save_weights('model_weights\efficientNet_weights/')